<a href="https://www.kaggle.com/code/avtnshm/clinical-modernbert-v-biomedicalmb-on-ddxplus-data?scriptVersionId=287761334" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Step 1: Load Data

import pandas as pd
import numpy as np
import json

train_df = pd.read_csv('/kaggle/input/mldataset/ddxplus/train.csv')
test_df = pd.read_csv('/kaggle/input/mldataset/ddxplus/test.csv')
validate_df = pd.read_csv('/kaggle/input/mldataset/ddxplus/validate.csv')

print(train_df.shape, validate_df.shape, test_df.shape)


(1025602, 6) (132448, 6) (134529, 6)


In [2]:
with open('/kaggle/input/mldataset/ddxplus/release_evidences.json') as f:
    evidences = json.load(f)

evidences_df = pd.DataFrame.from_dict(evidences, orient='index')


In [3]:
def codes_to_text(evidence_str):
    if not isinstance(evidence_str, str):
        return ""
    parts = [c.strip() for c in evidence_str.split(";")]
    mapped = [evidences_df.loc[p, "question_en"] for p in parts if p in evidences_df.index]
    return " ".join(mapped)


In [4]:
train_df["TEXT"] = train_df["EVIDENCES"].apply(codes_to_text)
validate_df["TEXT"] = validate_df["EVIDENCES"].apply(codes_to_text)
test_df["TEXT"] = test_df["EVIDENCES"].apply(codes_to_text)


In [5]:
from sklearn.model_selection import train_test_split

train_30, _ = train_test_split(
    train_df,
    test_size=0.70,
    stratify=train_df["PATHOLOGY"],
    random_state=42
)

valid_30, _ = train_test_split(
    validate_df,
    test_size=0.70,
    stratify=validate_df["PATHOLOGY"],
    random_state=42
)

test_30, _ = train_test_split(
    test_df,
    test_size=0.70,
    stratify=test_df["PATHOLOGY"],
    random_state=42
)

print(train_30.shape, valid_30.shape, test_30.shape)


(307680, 7) (39734, 7) (40358, 7)


In [6]:
!pip install -q transformers
import torch
from transformers import AutoTokenizer, AutoModel

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using:", device)

cmb_model = AutoModel.from_pretrained('Simonlee711/Clinical_ModernBERT').to(device)
cmb_tok = AutoTokenizer.from_pretrained('Simonlee711/Clinical_ModernBERT')

bmb_model = AutoModel.from_pretrained("thomas-sounack/BioClinical-ModernBERT-base").to(device)
bmb_tok = AutoTokenizer.from_pretrained("thomas-sounack/BioClinical-ModernBERT-base")


Using: cuda


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/547M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at Simonlee711/Clinical_ModernBERT and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

In [7]:
def embed(text_list, tokenizer, model, batch_size=32):
    all_vecs = []

    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)

        with torch.no_grad():
            out = model(**inputs).last_hidden_state[:,0,:]

        all_vecs.append(out.cpu().numpy())

    return np.vstack(all_vecs)


In [8]:
train_texts = train_30["TEXT"].tolist()
valid_texts = valid_30["TEXT"].tolist()
test_texts  = test_30["TEXT"].tolist()

cmb_train = embed(train_texts, cmb_tok, cmb_model)
bmb_train = embed(train_texts, bmb_tok, bmb_model)


W1222 06:56:22.978000 38 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode


In [9]:
cmb_valid = embed(valid_texts, cmb_tok, cmb_model)
bmb_valid = embed(valid_texts, bmb_tok, bmb_model)

cmb_test = embed(test_texts, cmb_tok, cmb_model)
bmb_test = embed(test_texts, bmb_tok, bmb_model)


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

y_train = train_30["PATHOLOGY"]
y_valid = valid_30["PATHOLOGY"]

clf_cmb = LogisticRegression(max_iter=200)
clf_cmb.fit(cmb_train, y_train)

pred_cmb = clf_cmb.predict(embed(valid_texts, cmb_tok, cmb_model))
print("CMB Macro F1:", f1_score(y_valid, pred_cmb, average='macro'))


CMB Macro F1: 0.0025040750277574354


In [11]:
clf_bmb = LogisticRegression(max_iter=200)
clf_bmb.fit(bmb_train, y_train)

pred_bmb = clf_bmb.predict(embed(valid_texts, bmb_tok, bmb_model))
print("BMB Macro F1:", f1_score(y_valid, pred_bmb, average='macro'))


BMB Macro F1: 0.0025040750277574354


In [12]:
from sklearn.metrics import top_k_accuracy_score

pred_probs_cmb = clf_cmb.predict_proba(cmb_valid)
pred_probs_bmb = clf_bmb.predict_proba(bmb_valid)

print("CMB Top-3:", top_k_accuracy_score(y_valid, pred_probs_cmb, k=3))
print("BMB Top-3:", top_k_accuracy_score(y_valid, pred_probs_bmb, k=3))


CMB Top-3: 0.17974530628680727
BMB Top-3: 0.17974530628680727


In [13]:
from sklearn.neural_network import MLPClassifier

mlp_cmb = MLPClassifier(hidden_layer_sizes=(512,256), max_iter=20)
mlp_cmb.fit(cmb_train, y_train)
pred_cmb_mlp = mlp_cmb.predict_proba(cmb_valid)

print("CMB Top-3:", top_k_accuracy_score(y_valid, pred_cmb_mlp, k=3))
print("CMB Top-5:", top_k_accuracy_score(y_valid, pred_cmb_mlp, k=5))


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


CMB Top-3: 0.17974530628680727
CMB Top-5: 0.23695072230331707


In [15]:
mlp_bmb = MLPClassifier(hidden_layer_sizes=(256,), max_iter=5)
mlp_bmb.fit(bmb_train, y_train)
pred_bmb_mlp = mlp_bmb.predict_proba(bmb_valid)

print("BMB Top-3:", top_k_accuracy_score(y_valid, pred_bmb_mlp, k=3))
print("BMB Top-5:", top_k_accuracy_score(y_valid, pred_bmb_mlp, k=5))


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(


BMB Top-3: 0.17974530628680727
BMB Top-5: 0.23430814919212764


In [18]:
import shutil
shutil.rmtree("/kaggle/working", ignore_errors=True)
